In [49]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats
import os
os.chdir('C:\\Users\\Himanshu\\Desktop\\QuantiPy')
from utils.db.fetch import fetch_entries



#df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df = fetch_entries(market_name='indian_equity', timeframe='1d', all_entries=True)
df = df['^NSEI']
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)

df['EMA'] = df['ema_100']
df.tail()

df

indicator_name,timestamp,open,high,low,close,volume,average_volume_90,ema_100,ema_200,gap_90_0.15,slope_r2_product_15,slope_r2_product_30,slope_r2_product_90,spike_90_0.15,supertrend_10_3.0,EMA
0,2014-09-04 00:00:00,8114.200195,8114.799805,8060.899902,8095.950195,8095.950195,0.000000e+00,8095.950195,8095.950195,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,8095.950195
1,2014-09-05 00:00:00,8099.899902,8122.700195,8049.850098,8086.850098,8086.850098,0.000000e+00,8095.769995,8095.859647,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,8095.769995
2,2014-09-08 00:00:00,8132.950195,8180.200195,8126.149902,8173.899902,8173.899902,0.000000e+00,8097.317122,8096.636167,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,8097.317122
3,2014-09-09 00:00:00,8161.899902,8174.549805,8126.500000,8152.950195,8152.950195,0.000000e+00,8098.418767,8097.196506,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,8098.418767
4,2014-09-10 00:00:00,8135.549805,8135.750000,8082.100098,8094.100098,8094.100098,0.000000e+00,8098.333249,8097.165696,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,8098.333249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,2024-08-26 00:00:00,24906.099609,25043.800781,24874.699219,25010.599609,25010.599609,5.543968e+08,23564.649869,22516.026689,0.0,0.001530,7.944671e-09,0.000858,0.0,24299.788512,23564.649869
2448,2024-08-27 00:00:00,25024.800781,25073.099609,24973.650391,25017.750000,25017.750000,5.558393e+08,23593.424129,22540.919458,0.0,0.001586,8.440541e-07,0.000862,0.0,24400.024895,23593.424129
2449,2024-08-28 00:00:00,25030.800781,25129.599609,24964.650391,25052.349609,25052.349609,5.573625e+08,23622.313742,22565.908813,0.0,0.001550,5.815930e-06,0.000863,0.0,24436.625140,23622.313742
2450,2024-08-29 00:00:00,25035.300781,25192.900391,24998.500000,25151.949219,25151.949219,5.590159e+08,23652.603554,22591.640559,0.0,0.001787,2.070316e-05,0.000861,0.0,24487.930204,23652.603554


In [8]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [9]:
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [10]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [48]:
dfpl = df[1570:1633]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=15, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [44]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    pattern_detected = False

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition1 = (idxhighs[0] < idxlows[0] 
                           < idxhighs[1] < idxlows[1] 
                           < idxhighs[2]< idxlows[2])
        
        order_condition2 = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2]< idxhighs[2])
        
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        pattern_1 = (lows[0] < highs[0] and
            lows[1] > lows[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] > highs[1] and highs[2] > lows[2]
            )

        pattern_2 = (lows[0] < highs[0] and
            lows[1] < lows[0] and
            highs[1] < highs[0] and
            lows[2] < lows[1] and
            highs[2] > highs[1] 
            )

        if ( (order_condition1 and
            (pattern_1)
        ) or (order_condition2 and
            (pattern_2)
        )
        ):
            pattern_detected = True

    if pattern_detected:
        return 1
    else:
        return 0

In [45]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

In [46]:
df[df['pattern_detected']!=0]

indicator_name,timestamp,open,high,low,close,volume,average_volume_90,ema_100,ema_200,gap_90_0.15,slope_r2_product_15,slope_r2_product_30,slope_r2_product_90,spike_90_0.15,supertrend_10_3.0,EMA,EMASignal,isPivot,pointpos,pattern_detected
1050,2018-12-20 00:00:00,10885.200195,10962.549805,10880.049805,10951.700195,10951.700195,1.190045e+08,10773.679145,10702.963018,0.0,0.000049,0.000254,-0.000319,0.0,10590.331506,10773.679145,0,0,NaN,1
1051,2018-12-21 00:00:00,10944.250000,10963.650391,10738.650391,10754.000000,10754.000000,1.188482e+08,10773.289459,10703.470848,0.0,0.000104,0.000229,-0.000303,0.0,10590.331506,10773.289459,0,0,NaN,1
1052,2018-12-24 00:00:00,10780.900391,10782.299805,10649.250000,10663.500000,10663.500000,1.186549e+08,10771.115411,10703.073128,0.0,0.000086,0.000155,-0.000288,0.0,10590.331506,10771.115411,0,0,NaN,1
1053,2018-12-26 00:00:00,10635.450195,10747.500000,10534.549805,10729.849609,10729.849609,1.184722e+08,10770.298266,10703.339561,0.0,0.000129,0.000092,-0.000269,0.0,10590.331506,10770.298266,0,2,10534.548805,1
1633,2021-05-10 00:00:00,14928.250000,14966.900391,14892.500000,14942.349609,14942.349609,2.146079e+08,14276.694148,13446.922610,0.0,0.000901,0.000002,0.000023,0.0,15111.717260,14276.694148,2,0,NaN,1
